## Setup

In [1]:
import numpy as np
rng = np.random.default_rng()  
import matplotlib.pylab as plt

## Helpful recources

https://hef.ru.nl/~tbudd/mct/lectures/markov_chain_monte_carlo.html

https://hef.ru.nl/~tbudd/mct/lectures/cluster_algorithms.html

## The 2D Ising model
The Ising model is a classical way of describing ferromagnetism using simplified spin-spin interactions. The 2D version of the Ising model considers an $w$ by $w$ square lattice, resulting in $N=w^2$ spin sites. If the lattice is given periodic boundary conditions, as is conventional, each spin will have exactly 4 neighbouring spins. The energy of the system is then given by

$ E = -J \sum_i \sum_j s_i s_j - \mu H \sum^{N}_{i=1} s_i $

where we sum over all spins $s_i$ and their neighbours $s_j$ with exchange energy $J$, magnetic moment $\mu$, and externally applied magnetic field $H$.

The most interesting variable of the system is the relative magnetisation of the complete lattice 

$M = \frac{1}{N}\sum_i^N s_i$

or the expected total magnetisation

$\mathbb{E}[M(\mathbf{s})]=\sum_{s \in\{-1,1\}^N}M(s) p_{\mathbf{s}}(s)$

with $p_{\mathbf{s}}(s)$ the probability mass function. The magnetisation shows a very strong dependence on the temperature as well as the externally applied magnetic field $H$. The magnetisation can also show a high variance. 

The analytical approach to calculate compute these expected values would be to use the probability mass function 

$p_{\mathbf{s}}(s)=\frac{1}{Z} e^{-\beta E}$

with partition function

$Z=\sum_{s \in\{-1,1\}^N} e^{-\beta E}$

and inverse temperature $\beta = \frac{1}{k_bT}$. However, computing this type of partition function quickly becomes arduous or simply infeasible for a lattice with any interesting number of spins, especially if the variance is also high. We thus typically use computational methods to investigate such systems and estimate the magnetisation $M$.

## A Monte Carlo model
The first steps of making the model will be to construct the lattice and several functions that compute essential properties of the system.

In [2]:
def init_lattice(width,type):
    '''Produce an initial lattice with spins. Type 1 gives all spins 1, type 0 gives a random spins, type -1 gives all spins -1. Captures invalid types.'''
    # TO DO

def neighbouring_sites(i,j,width):
    '''Return the coordinates of the 4 sites adjacent to [i,j] on an width*width lattice. Takes into account periodic boundary conditions.'''
    # TO DO

def neighbouring_spins_sum(i,j,lattice,width):
    '''Sums the spins of all neighbours of the spin at [i,j].'''
    # TO DO

def compute_magnetisation(lattice):
    '''Computes the magnetisation of the lattice.'''
    # TO DO (np.sum() may be useful)

def plot_lattice(lattice,ax,title):
    '''Plot the lattice configuration.'''
    ax.matshow(lattice, vmin=-1, vmax=1, cmap=plt.cm.binary)
    ax.title.set_text(title)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticks([])
    ax.set_xticks([])

## The Metropolis-Hastings algorithm
In order to be physically accurate, the transitions rates in our system have to satisfy detailed balance. The Metropolis-Hastings algorithm provides us with a way to select such transition rates without a need for knowing the normalisation of the probability distribution. 

$P(s_i \rightarrow -s_i) = \min \left(1, e^{-(\beta E_{-s_i}-\beta E_{s_i})}\right) = \min \left(1, e^{-\beta \Delta E}\right) = \begin{cases}1 & \text { if } \Delta E \leq 0 \\ e^{-\beta\Delta E} & \text { if } \Delta E > 0 \end{cases}$

where $\Delta E$ is the change in energy of the total lattice.

$\Delta E = 2J s_i \sum_j s_j + 2 \mu H s_i $

With this we can set up way to evolve the system. For convenience, it may be useful to define the interaction energies in terms of $\beta J$ and $\beta \mu H$.

In [3]:
def compute_betaDeltaE(i,j,lattice,width,betaJ,betaMuH):
    '''Computes the energy difference between the old and new state if spin [i,j] would be flipped.'''
    # TO DO

def attempt_spin_flip(lattice,width,betaJ,betaMuH):
    '''Applies the Metropolis-Hastings algorithm to try and flip a spin.'''
    # TO DO

def evolve_and_plot(lattice,betaJ,betaMuH,plot_times):
    '''Evolves the lattice using the Metropolis-Hastings algorithm and plots the lattice at different times.'''
    # TO DO
    fig, ax = plt.subplots(1,len(plot_times),figsize=(12,4))
    for t in range(plot_times[-1]+1):
        # TO DO
        if t in plot_times:
            plot_lattice(lattice,ax[plot_times.index(t)],"t = {}".format(t))
    plt.show()

def evolve_and_compute_M(lattice,betaJ,betaMuH,avg_times):
    '''Evolves the lattice using the Metropolis-Hastings algorithm and returns the average magnetisation computed using different time steps.'''
    # TO DO

## Testing the model
Evolve the system and plot it at various time steps to see if it appears to behave properly. Let's first get a feel for what a random lattice really looks like. Consider a system with no coupling $\beta J = 0$ and external magnetic field $\beta \mu H = 0$. Try both starting with a random lattice and a lattice with all spins in one direction.

In [ ]:
# Testing the Metropolis-Hastings algorithm
plot_times = [0,10,100,1000,10000,100000]

# TO DO

## Limiting case 1: only an external field
When $beta J=0$ there is no coupling. In such a case, only thermal motion and the external magnetic field affect the flipping of the spins. Do a few simulations with $-3 < \beta \mu H < 3$ and observe what happends. Again try both starting with a random lattice and a lattice with all spins in one direction. Qualitatively explain the system behaviour.

In [ ]:
plot_times = [0,10,100,1000,10000,100000]

# TO DO

In the case of no coupling, an analytical expression for the mean magnetisation $\langle M\rangle$ exists. The expression is:

$\langle M\rangle = \tanh \left(\beta \mu H \right)$

Show that the Monte Carlo method correctly repoduces the above analytical formula.

In [ ]:
betaMuHs_analytical = np.linspace(-3,3,100) # betaMuH values to plot
M_analytical = # TO DO

avg_times = # Pick some suitable time points at which to sample your magnetisation. These values should be averaged to get a stable mean

# TO DO: Simulate the system at different magnetic field strengths

plt.xlabel ('Magnetic field strength betaMuH')
plt.ylabel ('Magnetisation M')
plt.legend(loc='upper left')
plt.show()

## Limiting case 2: only coupling

Now let us investigate coupling without an external magnetic field. Try the same as previously, using different starting conditions, but introduce some ferromagnetic coupling ($\beta J = 0.2$) or anti-ferromagnetic coupling ($\beta J = -0.2$). Qualitatively explain the system behaviour.

In [ ]:
# TO DO


Also for this case, no field and only coupling, Lars Onsanger discovered an analytical solution:

$\langle M\rangle= \begin{cases}0 & \text { if } T \geq T_c \\ \pm \left(1-\sinh \left(2 \beta J\right)^{-4}\right)^{\frac{1}{8}} & \text { if } T<T_c\end{cases}$

where $T_c$ is the critical temperature given by

$T_c=\frac{2 J}{k_b \ln (1+\sqrt{2})}$

Show again that the Monte Carlo method correctly repoduces the above analytical formula. What do you notice about the system around the critical point?

Hint: The system will equilibrate much faster if you start with an initial state that is close to the expected final state.

In [ ]:
# TO DO

# Simuatle the system at different coupling strengths

plt.xlabel ('Coupling strength betaJ')
plt.ylabel ('Magnetisation M')
plt.legend(loc='upper left')
plt.show()

## Optional: implement the Wolff algorithm
**One more non-optional exercise after this, please look ahead.** In some cases, as you might realise in the next exercise, it can take quite long for the Metropolis-Hastings algorithm to sample the state space of the system. The Wolff algorithm was developed to speed up the exploration of the state space by flipping multiple spins (of the same sign) at once.

The algorithm works as follows:

- Select a spin $s_i$ at random

- Start checking all neighbouring spins $s_j$

- Add a neighbouring spin $s_j$ to the cluster with probability $P_{add} = 1-\exp(-\beta \Delta E)$, but only if it has the same sign as $s_i$

- Repeat until no unvisited neighbours remain

- Flip all spins

Implement Wolff algorithm and compare the equilibration speed to that of the Metropolis-Hastings algorithm for a system with only coupling ($\beta J=0.2$). Hint: In practice it is more convenient to already flip all visited spins.

For the interested reader, an even more efficient method is the Swendsen-Wang algorithm (https://en.wikipedia.org/wiki/Swendsen%E2%80%93Wang_algorithm).

In [9]:
def spin_flip_wolff(lattice,width,betaJ,betaMuH):
    '''Applies the Wolff algorithm to try and flip a spin.'''
    i,j = rng.integers(0,width,2) # Select a random seed
    # TO DO
    unvisited = # TO DO
    while (len(unvisited)>0):   # while unvisited sites remain
        i,j = unvisited.pop(0)  # take one and remove from the unvisited list
        for x,y in neighbouring_sites(i,j,width):
            # TO DO
    return cluster_size

def evolve_and_plot_wolff(lattice,betaJ,betaMuH,plot_times):
    '''Evolves the lattice using the Wolff algorithm and plots the lattice at different times.'''
    # TO DO
    fig, ax = plt.subplots(1,len(plot_times),figsize=(12,4))
    for t in range(plot_times[-1]+1):
        # TO DO
        if t in plot_times:
            plot_lattice(lattice,ax[plot_times.index(t)],"t = {}".format(t))
    plt.show()

In [ ]:
# Testing the Wolff algorithm if it has been implemented

# TO DO

## Mandatory: Magnetic susceptibility
A common problem in physical simulations is that we are limited in the size of the systems that we can simulate within a reasonable time. Periodic boundary conditions are typically a good way of circumventing problems caused by finite systems, but some problems cannot be circumvented that easily. The magnetisation of the 2D Ising model shows a continuous phase transition, and statistical physics tells us the correlation length $\xi$ of the system thus follows the relation

$\xi \sim\left|T-T_c\right|^{-\nu}$

with $\nu$ is called the critical exponent. This means that the correlation length will quickly diverge near the critical point and correlate most, if not all, spins in the system with each other. This strongly affects the magnetic susceptibility $\chi$ that is defined as

$\chi=\frac{1}{N} \frac{\partial M}{\partial H}=\frac{\beta}{N}\left(\left\langle M^2\right\rangle-\langle M \rangle^2\right)$.

Run a number of tests to see how the magnetic susceptibility $\chi$ changes as the critical temperature of the system is approached. Repeat this for a smaller or larger lattice. What does this tell you about our ability to get accurate average properties for the system near the critical temperature?

Note: It is more convenient to use the definition of magnetic susceptibility that uses the variance of the magnetisation than the definition of the gradient w.r.t. the magnetic field.

In [11]:
def evolve_and_computeChi(lattice,betaJ,betaMuH,avg_times):
    '''Evolves the lattice using the Metropolis-Hastings algorithm and computes the average magnetisation using different time steps.'''
    # TO DO

In [ ]:
# TO DO
# Simulate the system at different coupling strengths (relative to the critical temperature)

plt.xlabel ('Coupling strength betaJ')
plt.ylabel ('Magnetic susceptibility Chi')
plt.show()

In [ ]:
# Smaller or larger lattice to show susceptibility changes

# TO DO
# Simulate the system at different coupling strengths (relative to the critical temperature)

plt.xlabel ('Coupling strength betaJ')
plt.ylabel ('Magnetic susceptibility Chi')
plt.show()

## Optional: Further assignments

**Investigate finite-size scaling of the lattice.** The critical temperature varies as a function of the lattice size $N=w*w$

$T_c(N) = T_c(\infty)+aN^{-1/b}$

with $a$ and $b$ being constants to estimate. Compare your estimated $T_c(\infty)$ to Onsanger's analytical result.

**Compute the decorrelation time of the magnetisation to determine the proper equilibration time.** The autovariance is defined as

$A(\tau)=\langle M'(t)M'(t+\tau) \rangle$

where $M'=M-\langle M \rangle$ and $\tau$ is the lag time. The autocorrelation is then given by $a(\tau)=\frac{A(\tau)}{A(0)}$. The decorrelation time can them be said to be $\tau$ for which $a(\tau)$ has dropped to $1/e$.

**Determine the heat capacity.** The fluctuation dissipation theorem states that the heat capacity of the system is given by

$C=\frac{\sigma_E}{k_B T^2} $

with $\sigma_E$ being the standard deviation in the system energy.